In [32]:
import pandas as pd
import numpy as np
import cPickle as pickle

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.preprocessing import Imputer

In [2]:
prop = pickle.load(open("../../datos/Data.p","rb"))

# Nos quedamos con la parte del set de datos que nos importa

In [3]:
p = prop[['property_type','place_name','surface_total_in_m2','surface_covered_in_m2',\
          'price_usd_per_m2','latlon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_LugaresTuristicos','Cant_ParadasTransporte','price_aprox_usd']]
p.dropna(inplace = True)

/home/facundo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


# Separamos la Latitud-Longitud

In [4]:
p['lat'] = p.apply(lambda row: row[5][0], axis=1)

/home/facundo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [5]:
p['lon'] = p.apply(lambda row: row[5][1], axis=1)

/home/facundo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [6]:
p = p[['property_type','place_name','surface_total_in_m2','surface_covered_in_m2',\
          'price_usd_per_m2','lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_LugaresTuristicos','Cant_ParadasTransporte','price_aprox_usd']]

# Asignamos un valor numerico al tipo de propiedad

In [7]:
def asignarPT( p ):
    if p == 'apartment':
        return 0
    if p == 'house':
        return 1
    if p == 'store':
        return 2
    return 3

In [8]:
p['property_type'] = p.apply(lambda x: asignarPT(x[0]), axis = 1)

# Asignamos un valor a los barrios

In [9]:
def asignarPV(name, dic):
    return dic[name]

In [10]:
hoodGroups = p[["place_name", "price_usd_per_m2"]]
hoodGroups = hoodGroups.groupby("place_name").agg([np.mean]).reset_index()
hoodGroups = hoodGroups.sort_values(by=("price_usd_per_m2", "mean"), ascending=False)

In [11]:
HGdics = {}
precio_ant = 0
val_ant = 0
for row in hoodGroups.iterrows():
    name = row[1][0]
    price = row[1][1]
    val = val_ant + (precio_ant/price)*10
    HGdics[name] = val
    precio_ant = price
    val_ant = val

In [12]:
p["place_value"] = p.apply(lambda x: asignarPV(x[1], HGdics), axis=1)

In [17]:
pE = p[['property_type','surface_total_in_m2',\
          'lat','lon','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'place_value', 'price_aprox_usd']]

# Cargo el Set de pruebas

In [16]:
proper = pickle.load(open("../../datos/propiedadesPruebaConColTransGas.p", "rb"))

In [37]:
p = proper[['property_type','place_name','surface_total_in_m2','surface_covered_in_m2',\
          'latlon','Cant_ColeYUniv','Cant_LocalesGastronomicos','Cant_ParadasTransporte']]

In [38]:
p['lat'] = p.apply(lambda row: row[4][0] if(np.all(pd.notnull(row[5]))) else row[5], axis=1)

p['lon'] = p.apply(lambda row: row[4][1] if(np.all(pd.notnull(row[5]))) else row[5], axis=1)

p = p[['property_type','surface_covered_in_m2',\
          'lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte']]

/home/facundo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/facundo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [39]:
p['property_type'] = p.apply(lambda x: asignarPT(x[0]), axis = 1)

# Prueba AdaBoost

In [40]:
Xtrn, Xtest, Ytrn, Ytest = train_test_split(pE[['property_type','surface_total_in_m2',
        'lat','lon','Cant_LocalesGastronomicos','Cant_ParadasTransporte',
        'place_value']],pE[['price_aprox_usd']],test_size=0.0)

In [41]:
estimador = GradientBoostingRegressor(n_estimators = 10, criterion ='mse', min_samples_split = 4, min_samples_leaf = 3, max_features = 'auto')
ab = AdaBoostRegressor(n_estimators=10, base_estimator = estimador)

ab.fit(Xtrn, Ytrn['price_aprox_usd'])

AdaBoostRegressor(base_estimator=GradientBoostingRegressor(alpha=0.9, criterion='mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3,
             max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=3, min_samples_split=4,
             min_weight_fraction_leaf=0.0, n_estimators=10, presort='auto',
             random_state=None, subsample=1.0, verbose=0, warm_start=False),
         learning_rate=1.0, loss='linear', n_estimators=10,
         random_state=None)

In [42]:
imp = Imputer(missing_values=np.nan, strategy='mean', axis=0)
imp = imp.fit(Xtrn)

In [43]:
pP = imp.transform(p)

In [46]:
p['price_aprox_usd'] = ab.predict(pP)

In [54]:
res = pd.DataFrame()

In [55]:
res['id'] = proper['id']

In [56]:
res['price_usd'] = p['price_aprox_usd']

In [57]:
res.to_csv(index=False, path_or_buf="resPhiAlpha.csv")